# ==============================================================================
# Project: FreshCart Customer Churn Prediction
# Goal: End-to-End Data Processing & Model Training Pipeline
#
# Purpose:
# This script consolidates all previous steps (Data Loading, Feature Engineering,
# and Modeling) into a single, reproducible pipeline. It simulates a production training run:
# 1. Load Raw Data
# 2. Apply Cutoff Strategy (Prevent Leakage)
# 3. Generate All Features (RFM + Behavioral + Advanced)
# 4. Train Final Model (using Optimized Hyperparameters)
# 5. Export Artifacts (Model & Metadata) for Deployment
# ==============================================================================

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import json
import sys
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report

In [2]:
# Add src to path to import custom modules
# Adjust the path if you are running this script from a different location
sys.path.append('../src') 
# If running from the root 'FreshCart-Churn-Prediction' folder, use: sys.path.append('src')

In [3]:
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, RANDOM_STATE
from data.data_loader import InstacartDataLoader

In [4]:
# Import Feature Engineering Modules
from features.rfm_features import RFMFeatureEngineer
from features.behavioral_features import BehavioralFeatureEngineer

print("✅ Environment Setup Complete")

✅ Environment Setup Complete


# ==============================================================================
# 📦 Step 1: Ingest Raw Data
# ==============================================================================

In [1]:
def load_data():
    print("⏳ Loading Raw Data...")
    loader = InstacartDataLoader(RAW_DATA_DIR)
    data = loader.load_all_data()

    orders_df = data['orders']
    products_df = data['products']
    order_products = pd.concat([
        data['order_products_prior'],
        data['order_products_train']
    ], ignore_index=True)

    print(f"✅ Data Loaded. Orders: {len(orders_df):,}, Products: {len(products_df):,}")
    return orders_df, products_df, order_products

# ==============================================================================
# 🛠️ Step 2: Feature Engineering Pipeline (The "Leakage-Free" Logic)
# ==============================================================================

In [2]:
def run_feature_pipeline(orders, order_products, products):
    """
    Executes the full feature engineering pipeline with strict time-based splitting.
    """
    print("\n⚙️ Starting Feature Pipeline...")
    
    # 1. SORT & SPLIT (Cutoff Strategy)
    print("   1. Applying Cutoff Strategy (Splitting History vs Future)...")
    orders_sorted = orders.sort_values(['user_id', 'order_number'])
    last_orders = orders_sorted.groupby('user_id').tail(1)  # Target
    orders_history = orders_sorted.drop(last_orders.index)  # Features History
    
    # Filter order_products for history only
    op_history = order_products[order_products['order_id'].isin(orders_history['order_id'])]
    
    # 2. GENERATE TARGETS
    print("   2. Generating Targets...")
    labels = last_orders[['user_id', 'days_since_prior_order']].copy()
    labels['is_churn'] = (labels['days_since_prior_order'] >= 30).astype(int)
    
    # 3. RFM FEATURES
    print("   3. Generating RFM Features...")
    rfm_eng = RFMFeatureEngineer()
    rfm_feats = rfm_eng.create_all_rfm_features(orders_history, op_history)
    
    # 4. BEHAVIORAL FEATURES
    print("   4. Generating Behavioral Features...")
    beh_eng = BehavioralFeatureEngineer()
    beh_feats = beh_eng.create_all_behavioral_features(orders_history, op_history, products)
    
    # 5. ADVANCED FEATURES (Derived)
    print("   5. Deriving Advanced Metrics (Velocity, Acceleration)...")
    # Purchase Velocity: 1 / (Average Days Between Orders + 1)
    rfm_feats['purchase_velocity'] = 1 / (rfm_feats['avg_days_between_orders'] + 1)
    
    # 6. MERGE ALL
    print("   6. Merging Feature Sets...")
    final_df = labels[['user_id', 'is_churn']].merge(rfm_feats, on='user_id', how='left')
    final_df = final_df.merge(beh_feats, on='user_id', how='left')
    
    # Fill NaNs with 0
    final_df = final_df.fillna(0)
    
    return final_df


# ==============================================================================
# 🤖 Step 3: Final Model Training
# ==============================================================================

In [3]:
def train_model(final_dataset):
    # 1. Prepare X and y
    feature_cols = [c for c in final_dataset.columns if c not in ['user_id', 'is_churn']]
    
    X = final_dataset[feature_cols]
    y = final_dataset['is_churn']

    # 2. Split for Final Validation
    # Ideally, retrain on full data for production, but keeping split for validation metrics
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    # 3. Load Best Hyperparameters
    try:
        with open(MODEL_DIR / 'best_params.json', 'r') as f:
            best_params = json.load(f)
        print("✅ Loaded Best Hyperparameters from previous step.")
    except:
        print("⚠️ Best params file not found. Using default parameters.")
        best_params = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'learning_rate': 0.05,
            'n_estimators': 1000,
            'scale_pos_weight': 2.3 # Imbalance handling
        }

    # 4. Train LightGBM
    print("\n🚀 Training Final LightGBM Model...")
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test, reference=dtrain)

    final_model = lgb.train(
        best_params,
        dtrain,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )

    print("✅ Model Training Complete.")
    return final_model, X_test, y_test, feature_cols


# ==============================================================================
# 📊 Step 4: Quick Validation & Sanity Check
# ==============================================================================

In [4]:
def validate_model(model, X_test, y_test):
    print("\n📊 Validating Model Performance...")
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob >= 0.38).astype(int) # Using our optimized threshold

    auc = roc_auc_score(y_test, y_pred_prob)
    print(f"Final AUC Score: {auc:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))


# ==============================================================================
# 💾 Step 5: Export Artifacts for Deployment
# ==============================================================================

In [5]:
def save_artifacts(model, feature_cols, final_dataset):
    print("\n📦 Exporting Production Artifacts...")

    # 1. Save Model
    model_path = MODEL_DIR / 'final_model_optimized.pkl'
    joblib.dump(model, model_path)
    print(f"  -> Model saved to: {model_path}")

    # 2. Save Feature List (Critical for API input validation)
    feature_path = PROCESSED_DATA_DIR / 'model_features.json'
    with open(feature_path, 'w') as f:
        json.dump(feature_cols, f)
    print(f"  -> Feature list saved to: {feature_path}")

    # 3. Save Dataset (Optional, for Dashboard EDA)
    data_path = PROCESSED_DATA_DIR / 'final_features_advanced.parquet'
    final_dataset.to_parquet(data_path)
    print(f"  -> Processed data saved to: {data_path}")

    print("\n🎉 PIPELINE FINISHED SUCCESSFULLY!")


# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

In [ ]:
if __name__ == "__main__":
    # 1. Load Data
    orders_df, products_df, order_products = load_data()
    
    # 2. Run Feature Pipeline
    final_dataset = run_feature_pipeline(orders_df, order_products, products_df)
    print(f"\n✅ Pipeline Complete. Dataset Shape: {final_dataset.shape}")
    
    # 3. Train Model
    final_model, X_test, y_test, feature_cols = train_model(final_dataset)
    
    # 4. Validate
    validate_model(final_model, X_test, y_test)
    
    # 5. Save
    save_artifacts(final_model, feature_cols, final_dataset)


⏳ Loading Raw Data...


NameError: name 'InstacartDataLoader' is not defined